# **Algorithms for Data Processing**


## **1. Data Cleaning**  
- Handling Missing Values:  
  - Mean/Median/Mode Imputation  
  - K-Nearest Neighbors (KNN) Imputation  
  - Interpolation  
- Outlier Detection and Removal:  
  - Z-score Method  
  - IQR (Inter-quartile Range) Method  
  - Isolation Forest  
  - Local Outlier Factor (LOF)  

In [60]:
import numpy as np
import pandas as pd

## **Mean Imputation**
Mean imputation is a technique used to handle missing values in a dataset by replacing them with the **mean** (average) of the available data for that feature. It is commonly applied to numerical data. The mean is calculated as the sum of all values divided by the number of values. This method assumes that the data is normally distributed and does not have significant skewness or outliers.

---

## **Median Imputation**
Median imputation replaces missing values with the **median** (middle value) of the available data for that feature. The median is the value that separates the higher half of the data from the lower half. This method is particularly useful for numerical data that is skewed or contains outliers, as the median is less sensitive to extreme values compared to the mean.

---

## **Mode Imputation**
Mode imputation is used to replace missing values with the **mode** (most frequent value) of the available data for that feature. This technique is typically applied to categorical data or discrete numerical data where a clear dominant value exists. The mode represents the value that appears most frequently in the dataset.

---


In [ ]:
# Mean Median Mode Imputations

# Sample data
data = {'A': [1, 2, np.nan, 4, 5]}
df = pd.DataFrame(data)

# Mean imputation
meanVal = df['A'].mean()
df['A']= df['A'].fillna(meanVal)
print("Mean:\n",df)

# Median Imputation
medianVal = df['A'].median()
df['A']= df['A'].fillna(medianVal)
print("Median:\n",df)

# Mode
modVal = df['A'].mod(0)
df['A']= df['A'].fillna(modVal)
print("Mode:\n",df)



### **K-Nearest Neighbors (KNN) Imputation**
KNN imputation is a technique used to handle missing values by replacing them with values from the **k-nearest neighbors** in the dataset. It leverages the idea that similar data points (rows) should have similar values for their features.

---

### **How KNN Imputation Works**
1. **Step 1: Identify Missing Values**  
   Locate the missing values in the dataset.

2. **Step 2: Compute Distances**  
   For each row with missing values, compute the distance (e.g., Euclidean, Manhattan) to all other rows using the available features.

3. **Step 3: Select Nearest Neighbors**  
   Identify the **k-nearest neighbors** (rows) with the smallest distances.

4. **Step 4: Impute Missing Values**  
   - For **numerical features**: Replace the missing value with the **mean** or **median** of the corresponding feature values from the k-nearest neighbors.
   - For **categorical features**: Replace the missing value with the **mode** (most frequent value) of the corresponding feature values from the k-nearest neighbors.

---

### **Key Parameters**
- **n_neighbors (k)**: The number of neighbors to consider. A smaller k may overfit, while a larger k may smooth out patterns.
- **Distance Metric**: Common metrics include Euclidean, Manhattan, or Minkowski distance.
- **Weights**: Neighbors can be weighted by their distance (closer neighbors have more influence).

---


### **Advantages of KNN Imputation**
- Preserves relationships between features.
- Works well for datasets with complex patterns.
- Flexible for both numerical and categorical data.

---

### **Disadvantages of KNN Imputation**
- Computationally expensive for large datasets.
- Sensitive to the choice of k and distance metric.
- Requires scaling of features for accurate distance calculations.

---

[Code](https://www.geeksforgeeks.org/k-nearest-neighbours/)


In [ ]:

def euclideanDistance(a, b):
    """Compute Euclidean distance between two vectors, ignoring NaNs."""
    mask = ~np.isnan(a) & ~np.isnan(b)  # Consider only non-NaN values
    if not np.any(mask):
        return np.inf  # If all values are NaN, return infinity
    return np.sqrt(np.sum((a[mask] - b[mask])**2))

def kNN(X, k=3):
    """KNN Imputation for missing values in a NumPy array."""
    xImputed = X.copy()  # Copy of original data
    nRows, nColumns = X.shape

    for rowIndex in range(nRows):
        for columnIndex in range(nColumns):
            if np.isnan(X[rowIndex, columnIndex]):  # Check for missing value
                distances = []
                
                # Find K nearest neighbors
                for neighborIndex in range(nRows):
                    if neighborIndex != rowIndex and not np.isnan(X[neighborIndex, columnIndex]):
                        dist = euclideanDistance(X[rowIndex], X[neighborIndex])
                        distances.append((dist, X[neighborIndex, columnIndex]))

                # Sort by distance and select K closest
                distances.sort(key=lambda x: x[0])
                kNeighbors = [val for _, val in distances[:k]]

                # Impute missing value with mean of K neighbors
                if kNeighbors:
                    xImputed[rowIndex, columnIndex] = np.mean(kNeighbors)

    return xImputed

# Example dataset with missing values (NaN)
X = np.array([[1.0, 2.0, np.nan],
              [2.0, np.nan, 3.0],
              [np.nan, 4.0, 5.0],
              [3.0, 4.0, 6.0]])

# Apply KNN imputation
X_imputed = kNN(X, k=2)

print("Original Data with Missing Values:")
print(X)

print("\nImputed Data:")
print(X_imputed)


### **Best Algorithm for Matrix Interpolation**  

Interpolation in a **matrix (2D data)** depends on the nature of missing values and the structure of the data. The best algorithm depends on:  

1. **Smoothness of Data**  
   - If data follows a smooth trend → **Spline Interpolation or Bicubic Interpolation**  
   - If data has sudden jumps → **Nearest-Neighbor Interpolation**  

2. **Computational Efficiency**  
   - If speed is important → **Linear Interpolation or Nearest-Neighbor**  
   - If accuracy is important → **Polynomial or Spline Interpolation**  

---

### **Common Matrix Interpolation Methods**  

| Algorithm | Description | Best For |
|-----------|-------------|----------|
| **Bilinear Interpolation** | Uses linear interpolation in both row and column directions | Image resizing, smooth surfaces |
| **Bicubic Interpolation** | Uses cubic polynomials for smoother results | High-quality image scaling |
| **Nearest-Neighbor Interpolation** | Uses the closest available value | Discrete data, quick estimation |
| **Spline Interpolation** | Fits smooth curves across the data points | Geospatial data, scientific applications |
| **Kriging Interpolation** | A geostatistical method that models spatial correlation | Geographic and environmental data |

---


### **Best Choice for Matrix Interpolation**
| **Scenario** | **Best Algorithm** | **Best Library** |
|-------------|----------------|---------------|
| **Image Processing** | Bicubic Interpolation | OpenCV (`cv2`) |
| **Smooth Data (Geospatial, Climate)** | Kriging, Spline | SciPy (`scipy.interpolate`) |
| **Quick Estimation** | Nearest-Neighbor | NumPy (`numpy.interp`) |
| **General Purpose** | Bilinear, Bicubic | SciPy (`griddata`) |

---


In [ ]:

def linearInterpolation(X):
    """Perform linear interpolation on 1D NumPy array with missing values (NaN)."""
    n = len(X)
    for i in range(n):
        if np.isnan(X[i]):  # If missing value found
            left, right = None, None

            # Find the nearest left non-NaN value
            for k in range(i - 1, -1, -1):
                if not np.isnan(X[k]):
                    left = (k, X[k])
                    break

            # Find the nearest right non-NaN value
            for k in range(i + 1, n):
                if not np.isnan(X[k]):
                    right = (k, X[k])
                    break

            # If both left and right exist, apply linear interpolation
            if left and right:
                x1, y1 = left
                x2, y2 = right
                X[i] = y1 + (y2 - y1) * (i - x1) / (x2 - x1)

    return X

def nearestNeighborInterpolation(X):
    """Perform nearest-neighbor interpolation on 1D NumPy array with missing values."""
    n = len(X)
    for i in range(n):
        if np.isnan(X[i]):  # If missing value found
            left, right = None, None

            # Find the nearest left non-NaN value
            for k in range(i - 1, -1, -1):
                if not np.isnan(X[k]):
                    left = X[k]
                    break

            # Find the nearest right non-NaN value
            for k in range(i + 1, n):
                if not np.isnan(X[k]):
                    right = X[k]
                    break

            # Use the nearest available value
            if left is not None and right is not None:
                X[i] = left if (i - k) < (k - i) else right
            elif left is not None:
                X[i] = left
            elif right is not None:
                X[i] = right

    return X

def polynomialInterpolation(X, degree=2):
    """Perform polynomial interpolation on a 1D NumPy array with missing values."""
    xKnown = np.where(~np.isnan(X))[0]  # Indices of known values
    yKnown = X[xKnown]  # Known values
    xMissing = np.where(np.isnan(X))[0]  # Indices of missing values

    # Fit a polynomial curve to the known data points
    polyCoefficients = np.polyfit(xKnown, yKnown, degree)
    poly_func = np.poly1d(polyCoefficients)

    # Predict missing values using the polynomial function
    X[xMissing] = poly_func(xMissing)

    return X

# Example dataset with missing values
X = np.array([1.0, np.nan, 3.0, 4.0, np.nan, 6.0, np.nan, 8.0])

# Apply interpolation methods
XLinear = linearInterpolation(X.copy())
XNearest = nearestNeighborInterpolation(X.copy())
XPoly = polynomialInterpolation(X.copy(), degree=2)

print("Original Data with Missing Values:")
print(X)

print("\nLinear Interpolation:")
print(XLinear)

print("\nNearest-Neighbor Interpolation:")
print(XNearest)

print("\nPolynomial Interpolation (Degree 2):")
print(XPoly)


In [ ]:
# Using Lib
x = np.array([0, 1, 2, 3, 4])
y = np.array([0, 1, np.nan, 3, 4])

# Interpolate missing value
yIntercept = np.interp(x, x[~np.isnan(y)], y[~np.isnan(y)])
print(yIntercept)

# Use Scipy for 2D and OpenCV for matrix - use cases are define above




# **Outlier Detection and Removal**  

### **What is an Outlier?**  
An **outlier** is a data point that significantly differs from the rest of the dataset. Outliers can occur due to errors, variability in data, or rare events. Detecting and removing outliers is essential in **data preprocessing** for improving machine learning model performance.  

---

## **Methods for Outlier Detection and Removal**  

### **1. Z-score Method (Standard Score Method)**  
 **Concept:**  
- Measures how many standard deviations a data point is from the mean.  
- If a data point's **Z-score is too high or too low**, it's considered an outlier.  

 **Formula:**  
$
Z = \frac{(X - \mu)}{\sigma}
$
Where:  
- $ X $ = data point  
- $ \mu $ = mean of the dataset  
- $ \sigma $ = standard deviation  

 **Threshold:**  
- Commonly used thresholds: **Z > 3 or Z < -3** (99.7% of data falls within 3 standard deviations).  

 **Steps:**  
1. Compute the mean ($\mu$) and standard deviation ($\sigma$).  
2. Calculate the Z-score for each data point.  
3. Remove points where **Z-score > threshold (usually 3 or -3)**.  

---

### **2. IQR (Interquartile Range) Method**  
 **Concept:**  
- Uses the **middle 50% of the data** to detect outliers.  
- Any value **below the lower bound or above the upper bound** is considered an outlier.  

 **Formula:**  
$
IQR = Q3 - Q1
$
$
\text{Lower Bound} = Q1 - 1.5 \times IQR
$
$
\text{Upper Bound} = Q3 + 1.5 \times IQR
$
Where:  
- **Q1 (25th percentile)** = First quartile  
- **Q3 (75th percentile)** = Third quartile  
- **IQR (Interquartile Range)** = Spread of middle 50% of data  

 **Steps:**  
1. Compute **Q1 (25th percentile)** and **Q3 (75th percentile)**.  
2. Compute **IQR = Q3 - Q1**.  
3. Compute **upper** and **lower bounds**.  
4. Remove values outside these bounds.  

---

### **3. Isolation Forest (IForest)**  
 **Concept:**  
- A machine learning algorithm that isolates anomalies by **randomly selecting features** and **splitting data points**.  
- Outliers get isolated faster because they lie in low-density regions.  

 **How It Works?**  
1. Randomly select a feature and split it at a random value.  
2. Build a tree where normal points need **more splits** to be isolated, while **outliers are isolated quickly**.  
3. Compute an **anomaly score**, where higher values indicate outliers.  

 **Pros:**  
 Works on **high-dimensional data**  
 **Unsupervised** (no need for labeled data)  
 Efficient for large datasets  


---

### **4. Local Outlier Factor (LOF)**  
 **Concept:**  
- Measures how **isolated a data point is** compared to its neighbors.  
- Uses **density comparison**—if a point is in a low-density region, it's an outlier.  

 **How It Works?**  
1. Compute the **local density** of each point by looking at its **k nearest neighbors**.  
2. Compare each point’s density with its neighbors.  
3. If a point’s density is **significantly lower** than its neighbors, it's an outlier.  

 **Pros:**  
 Works well in **clusters**  
 **Unsupervised** (no need for labeled data)  
 Detects **local anomalies** (useful when outliers are not globally different but locally different)  

---

## **Comparison of Outlier Detection Methods**  

| Method | Type | Best For | Pros | Cons |
|--------|------|----------|------|------|
| **Z-score** | Statistical | Normally distributed data | Simple, fast | Sensitive to skewed data |
| **IQR Method** | Statistical | Skewed data, small datasets | Robust to skewed data | Ignores data distribution |
| **Isolation Forest** | Machine Learning | Large, high-dimensional datasets | Works well on large datasets | Needs tuning |
| **LOF** | Machine Learning | Clustering-based outlier detection | Detects **local** anomalies | Computationally expensive |

---

## **Final Recommendation**
- **For Normally Distributed Data** → **Z-score**  
- **For Skewed Data / Small Data** → **IQR**  
- **For Large & High-Dimensional Data** → **Isolation Forest**  
- **For Clustered Data** → **Local Outlier Factor (LOF)**  


In [ ]:

# Sample dataset
data = np.array([10, 12, 14, 15, 16, 100])  # 100 is an outlier

# Compute mean and standard deviation
mean = np.mean(data)
standardDeviation = np.std(data)

# Compute Z-scores
zScores = (data - mean) / standardDeviation

# Remove outliers (Z > 3 or Z < -3)
filteredData = data[np.abs(zScores) < 3]
print(filteredData)




## ** Raw Implementation of Isolation Forest**

### **How It Works**
- Randomly selects a feature and a split value.
- Constructs a tree by recursively splitting the data.
- Outliers are isolated quickly in shallow trees.
- The average depth of a point determines its anomaly score.

---

## **Comparison of Isolation Forest vs Local Outlier Factor (LOF)**

| **Feature**             | **Isolation Forest**            | **Local Outlier Factor (LOF)**    |
|------------------------|---------------------------------|-----------------------------------|
| **Type**              | Tree-based method               | Density-based method             |
| **Best for**          | High-dimensional data           | Small and structured datasets    |
| **Computational Cost**| Fast (O(n log n))               | Slow (O(n²) for large datasets)  |
| **Interpretable**     | Yes (Tree splits)               | Hard to interpret densities      |
| **Works with Clusters** | No (assumes global outliers)  | Yes (detects local outliers)     |

---

## **Final Thoughts**
- **Use Isolation Forest** when working with **large, high-dimensional datasets**.
- **Use LOF** when outliers are **locally different from neighbors**.


In [ ]:

class IsolationTree:
    def __init__(self, maxDepth):
        self.maxDepth = maxDepth
        self.left = None
        self.right = None
        self.splitFeature = None
        self.splitValue = None
        self.size = 0

    def fit(self, X, depth=0):
        if depth >= self.maxDepth or X.shape[0] <= 1:
            return
        
        self.splitFeature = np.random.randint(0, X.shape[1])
        minValue, maxValue = np.min(X[:, self.splitFeature]), np.max(X[:, self.splitFeature])

        if minValue == maxValue:
            return

        self.splitValue = np.random.uniform(minValue, maxValue)
        
        leftMask = X[:, self.splitFeature] < self.splitValue
        xLeft, xRight = X[leftMask], X[~leftMask]

        self.left = IsolationTree(self.maxDepth)
        self.right = IsolationTree(self.maxDepth)

        self.left.fit(xLeft, depth + 1)
        self.right.fit(xRight, depth + 1)

class IsolationForest:
    def __init__(self, nTrees=100, maxDepth=10):
        self.nTrees = nTrees
        self.maxDepth = maxDepth
        self.trees = []

    def fit(self, X):
        self.trees = [IsolationTree(self.maxDepth) for _ in range(self.nTrees)]
        for tree in self.trees:
            sampleIndices = np.random.choice(X.shape[0], size=min(256, X.shape[0]), replace=False)
            tree.fit(X[sampleIndices])

    def pathLength(self, X, tree, depth=0):
        if tree is None or (tree.splitFeature is None and tree.splitValue is None):
            return depth

        if X[tree.splitFeature] < tree.splitValue:
            return self.pathLength(X, tree.left, depth + 1)
        else:
            return self.pathLength(X, tree.right, depth + 1)

    def anomalyScore(self, X):
        pathLengths = np.array([np.mean([self.pathLength(x, tree) for tree in self.trees]) for x in X])
        c = 2 * (np.log(X.shape[0] - 1) + 0.5772156649) - (2 * (X.shape[0] - 1) / X.shape[0])
        scores = 2 ** (-pathLengths / c)
        return scores

    def predict(self, X, threshold=0.6):
        scores = self.anomalyScore(X)
        return np.where(scores > threshold, -1, 1)  # -1 for outliers, 1 for normal points

# Example usage
X = np.array([[10], [12], [14], [15], [16], [100]])  # 100 is an outlier

model = IsolationForest(nTrees=50, maxDepth=8)
model.fit(X)

predictions = model.predict(X)
print(predictions)  # -1 indicates outlier, 1 indicates normal


In [68]:

class LocalOutlierFactor:
    def __init__(self, nNeighbors=3):
        self.nNeighbors = nNeighbors
        self.X = None

    def fit(self, X):
        self.X = X

    def euclideanDistance(self, a, b):
        return np.sqrt(np.sum((a - b) ** 2))

    def kDistance(self, point):
        distances = np.array([self.euclideanDistance(point, x) for x in self.X])
        sortedDistances = np.sort(distances)
        return sortedDistances[self.nNeighbors]

    def reachabilityDistance(self, point, neighbor):
        return max(self.kDistance(neighbor), self.euclideanDistance(point, neighbor))

    def localReachabilityDensity(self, point):
        distances = np.array([self.reachabilityDistance(point, neighbor) for neighbor in self.X])
        return 1 / (np.mean(distances) + 1e-10)  # Avoid division by zero

    def localOutlierFactor(self, point):
        lrd_point = self.localReachabilityDensity(point)
        lrd_neighbors = np.array([self.localReachabilityDensity(neighbor) for neighbor in self.X])
        return np.mean(lrd_neighbors) / (lrd_point + 1e-10)  # Avoid division by zero

    def predict(self, X, threshold=1.5):
        lof_scores = np.array([self.localOutlierFactor(point) for point in X])
        return np.where(lof_scores > threshold, -1, 1)  # -1 for outliers, 1 for normal points

# Example usage
X = np.array([[10], [12], [14], [15], [16], [100]])  # 100 is an outlier

lof_model = LocalOutlierFactor(nNeighbors=2)
lof_model.fit(X)

predictions = lof_model.predict(X)
print(predictions)  # -1 indicates outlier, 1 indicates normal


[ 1  1  1  1  1 -1]
